In [1]:
import numpy as np
import pandas as pd
from collections import OrderedDict, defaultdict
import glob
import sys 
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
import gezi
from gezi import tqdm

In [2]:
d = pd.read_csv('../input/user_action2.csv')
d = d[d.is_first == 1]
d.feedid = d.feedid.astype(int)
d.userid = d.userid.astype(int)
d.date_ = d.date_.astype(int)

In [3]:
d.head()

,userid,feedid,date_,device,read_comment,comment,like,play,stay,click_avatar,forward,follow,favorite,finish_rate,stay_rate,is_first
0,8,71474,1,1,0,0,1,500,5366,0,0,0,0,0.045455,0.487818,1
1,131440,27529,1,2,0,0,0,44565,44823,0,0,0,0,0.768362,0.772810,1
2,131440,65100,1,2,0,0,0,14749,14992,0,0,0,0,0.983267,0.999467,1
3,131440,50282,1,2,0,0,0,3071,3361,0,0,0,0,0.099065,0.108419,1
4,205838,66594,1,2,0,0,0,15390,15618,0,0,0,0,1.183846,1.201385,1


In [4]:
# get history
# TODO add dislike ? no action and finish rate < 0.1 and stay i
# play time
# staty time as history, play time >  60s play time < 5s like this
# TODO make scripts and parallel base on user
# 历史收益比较大 尝试更多的可能历史

# 引入用户最近观看历史 50 个 ？    每个配合 各种action emb
# latest history  综合表示用户近期历史 ？ TODO

# finish rate 正负 参数如何最好
# stay rate  
# 另外全局角度 actition rate, finish rate, stay rate

ACTIONS = [
  'read_comment',
  'comment',
  'like',
  'click_avatar',
  'forward',
  'follow',
  'favorite'
]

history = {
  'action': {},
  'finish': {},
  'stay': {},
  'neg': {},
  'unfinish': {},
  'unstay': {},
  'latest': {}
}

for action in ACTIONS:
  history[action] = {}

userids = [int(x.strip().split()[0]) for x in open('../input/user_vocab.txt').readlines()]
for userid in userids:
  history['action'][userid] = []
  history['finish'][userid] = []
  history['stay'][userid] = []
  history['neg'][userid] = []
  history['unfinish'][userid] = []
  history['unstay'][userid] = []
  history['latest'][userid] = []
  for action in ACTIONS:
    history[action][userid] = []

def is_neg(row, play=None):
  for action in ACTIONS:
    if row[action] > 0:
      return False

  return True


d = d.sort_values(['date_'], ascending=False)
for _, row in tqdm(d.iterrows(), total=len(d), desc='user_action'):
  feedid = int(row['feedid'])
  userid = int(row['userid'])
  day = int(row['date_'])
  feedid = (feedid, day)
  history['latest'][userid].append(feedid)
  if row['finish_rate'] > 0.99:
    history['finish'][userid].append(feedid)
  if row['finish_rate'] < 0.01:
    history['unfinish'][userid].append(feedid)
  if row['stay_rate'] > 1:
    history['stay'][userid].append(feedid)
  if row['stay_rate'] < 0.01:
    history['unstay'][userid].append(feedid)
  is_neg_row = is_neg(row)
  if not is_neg_row:
    history['action'][userid].append(feedid)
  else:
    history['neg'][userid].append(feedid)
  for action in ACTIONS:
    if row[action] > 0:
      history[action][userid].append(feedid)

user_action:   0%|          | 0/7197989 [00:00<?, ?it/s]

In [5]:
gezi.save_pickle(history, '../input/history2.pkl')

In [6]:
history['read_comment'][131440]

[(58224, 14),
 (23168, 14),
 (75272, 14),
 (87099, 14),
 (3479, 14),
 (76128, 11),
 (23783, 10),
 (68332, 10),
 (74292, 10),
 (93184, 10),
 (96506, 10),
 (109497, 10),
 (61857, 9),
 (82488, 9),
 (50755, 8),
 (67486, 5),
 (69183, 5),
 (48522, 5)]

In [7]:
m = {
  'userid': [],
  'action': [],
  'action_len': [],
  'finish': [],
  'finish_len': [],
  'stay': [],
  'stay_len': [],
  'neg': [],
  'neg_len': [],
  'unfinish': [],
  'unfinish_len': [],  
  'unstay': [],
  'unstay_len': [], 
  'latest': [],
  'latest_len': [],
}
OTHER_ACTIONS = ['action', 'neg', 'finish', 'unfinish', 'stay', 'unstay', 'latest']
for action in ACTIONS:
  m[action] = []
  m[f'{action}_len'] = []

for userid in tqdm(userids):
  m['userid'].append(userid)
  for action in ACTIONS + OTHER_ACTIONS:
    m[action].append(' '.join(map(str, history[action][userid])))
    m[f'{action}_len'].append(len(history[action][userid]))

  0%|          | 0/20000 [00:00<?, ?it/s]

In [8]:
his_df = pd.DataFrame(m)

In [9]:
his_df.head()

,userid,action,action_len,finish,finish_len,stay,stay_len,neg,neg_len,unfinish,...,like,like_len,click_avatar,click_avatar_len,forward,forward_len,follow,follow_len,favorite,favorite_len
0,32579,"(48066, 14) (54970, 13) (89199, 13) (54420, 13...",26,"(89046, 14) (75816, 14) (24865, 14) (38476, 14...",131,"(89046, 14) (76147, 14) (77378, 14) (75816, 14...",148,"(64919, 14) (14858, 14) (99913, 14) (53178, 14...",945,"(77378, 14) (38374, 14) (14894, 14) (7139, 14)...",...,"(48066, 14) (54970, 13) (89199, 13) (54420, 13...",25,,0,,0,,0,,0
1,9541,"(79433, 14) (76562, 11) (71768, 10) (4244, 10)...",9,"(51707, 14) (21838, 14) (33011, 14) (74943, 14...",643,"(51707, 14) (21838, 14) (33011, 14) (74943, 14...",653,"(51707, 14) (58648, 14) (24534, 14) (21838, 14...",922,"(105402, 14) (88217, 14) (18624, 11) (65598, 1...",...,"(79433, 14) (4244, 10) (46192, 7) (46423, 6)",4,,0,,0,,0,,0
2,69134,"(42285, 6) (23241, 6) (97497, 4) (21926, 2) (7...",8,"(109421, 14) (72355, 14) (38433, 14) (75403, 1...",134,"(109421, 14) (72355, 14) (38433, 14) (75403, 1...",151,"(37618, 14) (55154, 14) (104924, 14) (109421, ...",951,"(104800, 14) (101823, 3) (102787, 2) (24836, 2)",...,"(42285, 6) (23241, 6) (97497, 4) (21926, 2) (7...",8,,0,"(21926, 2)",1,,0,,0
3,239494,"(15599, 13) (38109, 13) (21678, 10) (111818, 1...",17,"(28052, 14) (32987, 14) (42595, 14) (59901, 14...",299,"(28052, 14) (32987, 14) (42595, 14) (59901, 14...",308,"(27480, 14) (106450, 14) (74235, 14) (24881, 1...",938,"(24881, 14) (45486, 14) (82652, 14) (83981, 14...",...,"(38109, 13) (21678, 10) (100968, 8) (109457, 5...",6,,0,"(15599, 13) (111818, 10) (92806, 8) (105989, 5...",7,,0,,0
4,246409,"(83981, 14) (107275, 13) (67490, 13) (169, 13)...",14,"(57114, 14) (83981, 14) (111736, 14) (62257, 1...",415,"(57114, 14) (83981, 14) (111736, 14) (62257, 1...",472,"(57114, 14) (96948, 14) (111736, 14) (62257, 1...",940,"(31888, 9) (82869, 8) (25434, 8) (109045, 8) (...",...,"(107275, 13) (67490, 13) (169, 13) (110733, 10...",10,,0,,0,,0,"(83981, 14) (72353, 4)",2


In [10]:
his_df.describe(percentiles=[.25,.5,.75,.9,.95,.99, .999])

,userid,action_len,finish_len,stay_len,neg_len,unfinish_len,unstay_len,latest_len,read_comment_len,comment_len,like_len,click_avatar_len,forward_len,follow_len,favorite_len
count,20000.00000,20000.000000,20000.000000,20000.000000,20000.00000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,125084.40225,25.204500,164.673450,189.123850,334.69495,47.243450,0.719450,359.899450,12.797250,0.148000,9.425050,2.742950,1.395450,0.263250,0.490550
std,72559.17173,46.703485,140.163164,150.941202,232.18396,83.615123,1.470017,240.088433,41.258482,1.674765,23.145271,5.711533,4.603274,1.219969,5.185765
min,8.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,61268.00000,5.000000,52.000000,67.000000,136.00000,7.000000,0.000000,153.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,125894.50000,10.000000,124.000000,147.000000,281.00000,21.000000,0.000000,308.000000,0.000000,0.000000,4.000000,1.000000,0.000000,0.000000,0.000000
75%,188072.25000,24.000000,242.000000,277.000000,502.00000,53.000000,1.000000,537.000000,5.000000,0.000000,8.000000,3.000000,1.000000,0.000000,0.000000
90%,225922.40000,58.000000,371.000000,412.000000,693.00000,109.100000,2.000000,731.000000,31.000000,0.000000,21.000000,8.000000,4.000000,1.000000,0.000000
95%,237992.95000,102.000000,449.000000,495.000000,780.00000,165.000000,3.000000,815.000000,73.000000,1.000000,36.000000,12.000000,7.000000,1.000000,2.000000
99%,247812.08000,243.000000,585.010000,631.000000,879.00000,473.010000,6.000000,901.000000,216.010000,3.000000,104.000000,27.000000,19.000000,5.000000,10.000000


In [11]:
his_df.to_csv('../input/history2.csv', index=False)